In [1]:
# %%writefile ../alignment_to_database.py

# """
# Script aims to read in the reference_dataframe file, select a taxonomic
# level and group, and read the path to the location of that data. It then
# prepares data for machine learning by converting base pair coding to numerical
# encoding, pads it out and then runs the algorithm
# """

# import pandas as pd
# from Bio import SeqIO
# import numpy as np
# import os
# import random
# import argparse
# import math
# import matplotlib.pyplot as plt
# import pandas as pd

# parser = argparse.ArgumentParser(description="""
# Script aims to read in the reference_dataframe file, and for each
# sample, randomly subsample n_reads number of reads and save the
# file, and save the keys used as as separate file. The script ends there.

# Outside the script these files will be utilised in minimap2 to
# map against a specified database, produce an output paf file
# and then remove the file to conserve space. 
# """)
# parser.add_argument("ref_df_fn", help="File path to the reference dataframe")
# parser.add_argument("data_root", help="Root folder for analysis/")
# parser.add_argument("--n_reads", "-c", help="count of reads per class")
# parser.add_argument("--d_type", "-d", help="type of database - CUSTOM or UNITE")
# group = parser.add_mutually_exclusive_group()
# group.add_argument("--verbose", "-v", "--v", action="store_true")
# group.add_argument("--quiet", "-q", "--q", action="store_true")
# args = parser.parse_args()

# # assign required arguments to variables
# ref_df_fn = args.ref_df_fn
# data_root = args.data_root

# # assign a number of reads per class
# n_reads = int(args.n_reads)

# # test to make sure both required file paths are input
# try:
#     os.path.exists(ref_df_fn)
# except:
#     print('Cannot find %s' % ref_df_fn)
# try:
#     os.path.exists(data_root)
# except:
#     print('Cannot find %s' % data_root)
    
# if args.verbose:
#     print('\033[1;34m' + "Reference dataframe is at " + ref_df_fn + '\033[0m')
#     print('\033[1;34m' + "Root directory is at " + data_root + '\033[0m')
#     print('\033[1;34m' + "Count of reads per sample is", n_reads,'\033[0m')
    
# # read in the reference dataframe from the argument path
# ref_df = pd.read_csv(ref_df_fn, index_col=None)

# # check whether the reference dataframe implies there are enough reads
# # to continue given n_reads
# try:
#     if ref_df[ref_df["# for use"] \
#               < n_reads].shape[0] > 0 :
#         print("These species need more reads.")
#         print(ref_df[ref_df["# for use"] \
#               < n_reads])
#         exit()
# except:
#     print('Check %s to have the wanted column names' % ref_df_fn)
   

# for index, row in ref_df.iterrows():
#     fasta_dict = SeqIO.to_dict(SeqIO.parse(row['path for use'], "fasta"))
#     key_list = []
#     for key in fasta_dict:
#         key_list.append(key)
#     print(len(key_list))
#     keys_list = random.sample(key_list,k=n_reads)
#     new_dict = {}
#     for key in keys_list:
#         new_dict[key] = fasta_dict[key]
#     if args.d_type.upper() == 'CUSTOM':
#         SeqIO.write(new_dict.values(), data_root+"database_mapping/custom/%s_%s.fasta" % (row['genus'],row['species']), "fasta")
#         with open(data_root+'database_mapping/custom/%s_%s_keys.csv'% (row['genus'],row['species']), 'w+') as f:
#             for key in keys_list:
#                 f.write("%s\n"%(key))
#     elif args.d_type.upper() == 'UNITE':
#         SeqIO.write(new_dict.values(), "analysis/database_mapping/unite/%s_%s.fasta" % (row['genus'],row['species']), "fasta")
#         with open(data_root+'database_mapping/unite/%s_%s_keys.csv'% (row['genus'],row['species']), 'w+') as f:
#             for key in keys_list:
#                 f.write("%s\n"%(key))

In [2]:
# %%writefile ../custom_minimap_result.py

# import Bio
# from Bio import SeqIO
# from Bio.Seq import Seq
# from Bio.SeqRecord import SeqRecord as SR
# from Bio.Blast import NCBIXML
# from Bio import pairwise2
# from Bio.pairwise2 import format_alignment
# from Bio import AlignIO
# from Bio.Align import AlignInfo
# import numpy as np
# import pandas as pd
# from pandas import DataFrame as df
# # import seaborn as sns
# import csv
# import matplotlib
# import matplotlib.pyplot as plt
# import subprocess
# import os
# from shutil import copy
# import random
# import warnings
# from Bio import pairwise2
# from Bio.pairwise2 import format_alignment
# # from ipysankeywidget import SankeyWidget
# warnings.filterwarnings("ignore")
# import argparse


# parser = argparse.ArgumentParser(description="""
# This program extracts a specified number of reads from a fasta
# file and saves them to a new fasta file
# """)
# group = parser.add_mutually_exclusive_group()
# group.add_argument("--verbose", "-v", "--v", action="store_true")
# group.add_argument("--quiet", "-q", "--q", action="store_true")
# parser.add_argument("input_file", help="The input file for extraction")
# args = parser.parse_args()

# if args.verbose:
#     print("Input file is " + args.input_file + "\n")

# f=open(args.input_file,"r")
# if f.mode == "r":
#     contents=f.read()

# tmp=contents.replace("\t",",").split('\n')
# tmp_dict = {}
# for line in tmp[:-1]:
#     tmp_dict[line.split(",")[0]] = str(line.split(",")[5])

# count_dict = {}
# correct = 0
# incorrect = 0
# for item in tmp_dict:
#     if tmp_dict[item] not in count_dict:
#         count_dict[tmp_dict[item]] = 1
#     else:
#         count_dict[tmp_dict[item]] = count_dict[tmp_dict[item]] + 1
        
# tmp = pd.DataFrame.from_dict(count_dict,orient='index',columns=["Count"])
# tmp.index.names = [args.input_file[33:-4]]
# tmp = tmp.sort_values(by="Count",ascending=False)

# if args.verbose:
#     print(tmp)

# tmp.to_csv(args.input_file[:-4]+'_match_distribution.csv',sep=',')

Starting points from Tavish  

reference_dataframe at '/media/MassStorage/tmp/TE/honours/analysis/Stats/reference_dataframe.csv' 

custom_database at '/media/MassStorage/tmp/TE/honours/database/custom_database_labelled.fasta'

taxonomy_file at '/media/MassStorage/tmp/TE/honours/analysis/Stats/taxonomy_file.csv'

#### workflow

* Two databases
* subsample 15000 reads per each mock community species. Save those out.
* map reads against both databases with minimap safe out data in paf format.
* get best hit per species (see what this means while looking at the data).
* add the full taxonomy to each best match using the taxonomy file.
* summarize data at different taxonomic ranks for each species.
* pull this all together somehow (summary across all the samples? focus on species of interest e.g. deleted from analyis?)

In [32]:
from Bio import SeqIO
import os
import random
import subprocess
import pandas as pd

In [33]:
reference_dataframe_fn = os.path.abspath('/media/MassStorage/tmp/TE/honours/analysis/Stats/reference_dataframe.csv')
max_custom_database_fn = os.path.abspath('/media/MassStorage/tmp/TE/honours/database/custom_database_labelled.fasta')
taxonomy_file_fn = os.path.abspath('/media/MassStorage/tmp/TE/honours/analysis/Stats/taxonomy_file.csv')

In [34]:
#threads to use
threads = 6

In [35]:
INPUT_BASEDIR = os.path.abspath('/media/MassStorage/tmp/TE/honours')

OUT_DIR = os.path.abspath('../../analysis/Mapping')
if not os.path.exists(OUT_DIR):
    os.mkdir(OUT_DIR)

In [36]:
### list of species in the max database
max_species = ['Puccinia_striiformis-tritici',
             'Zymoseptoria_tritici',
             'Pyrenophora_tritici-repentis',
             'Fusarium_oxysporum',
             'Tuber_brumale',
             'Cortinarius_globuliformis',
             'Aspergillus_niger',
             'Clavispora_lusitaniae',
             'Kluyveromyces_unidentified',
             'Penicillium_chrysogenum',
             'Rhodotorula_mucilaginosa',
             'Scedosporium_boydii',
             'Blastobotrys_proliferans',
             'Debaryomyces_unidentified',
             'Galactomyces_geotrichum',
             'Kodamaea_ohmeri',
             'Meyerozyma_guillermondii',
             'Wickerhamomyces_anomalus',
             'Yamadazyma_mexicana',
             'Yamadazyma_scolyti',
             'Yarrowia_lipolytica',
             'Zygoascus_hellenicus',
             'Aspergillus_flavus',
             'Cryptococcus_zero',
             'Aspergillus_unidentified',
             'Diaporthe_CCL067',
             'Diaporthe_unidentified',
             'Oculimacula_yallundae-CCL031',
             'Oculimacula_yallundae-CCL029',
             'Dothiorella_vidmadera',
             'Quambalaria_cyanescens',
             'Entoleuca_unidentified',
             'Asteroma_CCL060',
             'Asteroma_CCL068',
             'Saccharomyces_cerevisiae',
             'Cladophialophora_unidentified',
             'Candida_albicans',
             'Candida_metapsilosis',
             'Candida_orthopsilosis',
             'Candida_parapsilosis',
             'Candida_unidentified',
             'Kluyveromyces_marxianus',
             'Pichia_kudriavzevii',
             'Pichia_membranifaciens']

In [37]:
###Removed from second test database
species_delete = ['Candida_orthopsilosis',
                 'Candida_metapsilosis',
                 'Aspergillus_niger']

In [38]:
###species to be searched against both databases
mock_community = ['Penicillium_chrysogenum',
 'Aspergillus_flavus',
 'Aspergillus_niger',
 'Pichia_kudriavzevii',
 'Pichia_membranifaciens',
 'Candida_albicans',
 'Candida_parapsilosis',
 'Candida_orthopsilosis',
 'Candida_metapsilosis']

# mock_community = ['Puccinia_striiformis-tritici',
#              'Zymoseptoria_tritici',
#              'Pyrenophora_tritici-repentis',
#              'Fusarium_oxysporum',
#              'Tuber_brumale',
#              'Cortinarius_globuliformis',
#              'Aspergillus_niger',
#              'Clavispora_lusitaniae',
#              'Kluyveromyces_unidentified',
#              'Penicillium_chrysogenum',
#              'Rhodotorula_mucilaginosa',
#              'Scedosporium_boydii',
#              'Blastobotrys_proliferans',
#              'Debaryomyces_unidentified',
#              'Galactomyces_geotrichum',
#              'Kodamaea_ohmeri',
#              'Meyerozyma_guillermondii',
#              'Wickerhamomyces_anomalus',
#              'Yamadazyma_mexicana',
#              'Yamadazyma_scolyti',
#              'Yarrowia_lipolytica',
#              'Zygoascus_hellenicus',
#              'Aspergillus_flavus',
#              'Cryptococcus_zero',
#              'Aspergillus_unidentified',
#              'Diaporthe_CCL067',
#              'Diaporthe_unidentified',
#              'Oculimacula_yallundae-CCL031',
#              'Oculimacula_yallundae-CCL029',
#              'Dothiorella_vidmadera',
#              'Quambalaria_cyanescens',
#              'Entoleuca_unidentified',
#              'Asteroma_CCL060',
#              'Asteroma_CCL068',
#              'Saccharomyces_cerevisiae',
#              'Cladophialophora_unidentified',
#              'Candida_albicans',
#              'Candida_metapsilosis',
#              'Candida_orthopsilosis',
#              'Candida_parapsilosis',
#              'Candida_unidentified',
#              'Kluyveromyces_marxianus',
#              'Pichia_kudriavzevii',
#              'Pichia_membranifaciens']

In [39]:
ref_df = pd.read_csv(reference_dataframe_fn)
ref_df['name_species'] = ref_df['genus'] +"_"+ ref_df['species']

In [40]:
ref_df.name_species.tolist()

['puccinia_striiformis-tritici',
 'zymoseptoria_tritici',
 'pyrenophora_tritici-repentis',
 'fusarium_oxysporum',
 'tuber_brumale',
 'cortinarius_globuliformis',
 'aspergillus_niger',
 'clavispora_lusitaniae',
 'kluyveromyces_unidentified',
 'penicillium_chrysogenum',
 'rhodotorula_mucilaginosa',
 'scedosporium_boydii',
 'blastobotrys_proliferans',
 'debaryomyces_unidentified',
 'galactomyces_geotrichum',
 'kodamaea_ohmeri',
 'meyerozyma_guillermondii',
 'wickerhamomyces_anomalus',
 'yamadazyma_mexicana',
 'yamadazyma_scolyti',
 'yarrowia_lipolytica',
 'zygoascus_hellenicus',
 'aspergillus_flavus',
 'cryptococcus_zero',
 'aspergillus_unidentified',
 'diaporthe_ccl067',
 'diaporthe_unidentified',
 'oculimacula_yallundae-ccl031',
 'oculimacula_yallundae-ccl029',
 'dothiorella_vidmadera',
 'quambalaria_cyanescens',
 'entoleuca_unidentified',
 'asteroma_ccl060',
 'asteroma_ccl068',
 'saccharomyces_cerevisiae',
 'cladophialophora_unidentified',
 'candida_albicans',
 'candida_metapsilosis',


In [41]:
new_db_fn = os.path.join(OUT_DIR, 'gsref.db.fasta')

new_db_list = []
old_db_list = []
for seq in SeqIO.parse(max_custom_database_fn, 'fasta'):
    old_db_list.append(seq.id)
    if seq.id.lower() in ref_df.name_species.tolist():
        #print(seq.id)
        seq.id = seq.name = seq.description = seq.id.lower()
        new_db_list.append(seq)
    else:
        print(seq.id)

In [42]:
if len(new_db_list) == len(old_db_list):
    SeqIO.write(new_db_list, new_db_fn, 'fasta')
else:
    print("please check!")

In [43]:
sub_db_fn = os.path.join(OUT_DIR, 'gsref.subdb.fasta')
sub_db_list = []
for seq in new_db_list:
    if seq.id not in [x.lower() for x in species_delete]:
        sub_db_list.append(seq)

In [44]:
if len(sub_db_list) + len(species_delete) == len(new_db_list):
    SeqIO.write(sub_db_list, sub_db_fn, 'fasta' )
else:
    print("please check!")

In [45]:
[x.id for x in sub_db_list]

['puccinia_striiformis-tritici',
 'zymoseptoria_tritici',
 'pyrenophora_tritici-repentis',
 'fusarium_oxysporum',
 'tuber_brumale',
 'cortinarius_globuliformis',
 'clavispora_lusitaniae',
 'kluyveromyces_unidentified',
 'penicillium_chrysogenum',
 'rhodotorula_mucilaginosa',
 'scedosporium_boydii',
 'blastobotrys_proliferans',
 'debaryomyces_unidentified',
 'galactomyces_geotrichum',
 'kodamaea_ohmeri',
 'meyerozyma_guillermondii',
 'wickerhamomyces_anomalus',
 'yamadazyma_mexicana',
 'yamadazyma_scolyti',
 'yarrowia_lipolytica',
 'zygoascus_hellenicus',
 'aspergillus_flavus',
 'cryptococcus_zero',
 'aspergillus_unidentified',
 'diaporthe_ccl067',
 'diaporthe_unidentified',
 'oculimacula_yallundae-ccl031',
 'oculimacula_yallundae-ccl029',
 'dothiorella_vidmadera',
 'quambalaria_cyanescens',
 'entoleuca_unidentified',
 'asteroma_ccl060',
 'asteroma_ccl068',
 'saccharomyces_cerevisiae',
 'cladophialophora_unidentified',
 'candida_albicans',
 'candida_parapsilosis',
 'candida_unidentified

In [46]:
mock_community = [x.lower() for x in mock_community]

In [47]:
mock_community

['penicillium_chrysogenum',
 'aspergillus_flavus',
 'aspergillus_niger',
 'pichia_kudriavzevii',
 'pichia_membranifaciens',
 'candida_albicans',
 'candida_parapsilosis',
 'candida_orthopsilosis',
 'candida_metapsilosis']

In [48]:
def subsamplereads(in_fn, out_fn, n_reads):
    command = F'reformat.sh samplereadstarget={n_reads} in={in_fn} out={out_fn}'
    out = subprocess.getstatusoutput(command)
    if out[0] == 0:
        print(F":)Completed {command}\n")
    else:
        print(F":(check one {command}!!\n")

In [49]:
n_reads = 15000

In [50]:
MC_READ_DIR = os.path.join(OUT_DIR, 'MC_READS')
if not os.path.exists(MC_READ_DIR):
    os.mkdir(MC_READ_DIR)

In [51]:
ref_df.columns

Index(['Unnamed: 0', 'species', 'genus', 'family', 'order', 'class', 'phylum',
       'kingdom', '# raw reads', '# reads after homology filtering',
       '# reads after length filtering', '# for use', 'path to raw reads',
       'path to homology filtering', 'path to length filtering',
       'path for use', 'name_species'],
      dtype='object')

In [52]:
fn_subsampling = {}
for x in mock_community:
    fn_subsampling[x] = (ref_df[(ref_df['species'] == x.split('_')[1]) & (ref_df['genus'] == x.split('_')[0])]['path for use'].tolist()[0])
    fn_subsampling[x] = os.path.join(INPUT_BASEDIR, fn_subsampling[x])
fn_subsampling

{'penicillium_chrysogenum': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171103_FAH15473/barcode10/length_restricted_for_use.fasta',
 'aspergillus_flavus': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171207_FAH18654/barcode12/length_restricted_for_use.fasta',
 'aspergillus_niger': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171103_FAH15473/barcode07/length_restricted_for_use.fasta',
 'pichia_kudriavzevii': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20180108_FAH18647/barcode11/length_restricted_for_use.fasta',
 'pichia_membranifaciens': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20180108_FAH18647/barcode12/length_restricted_for_use.fasta',
 'candida_albicans': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20180108_FAH18647/barcode03/length_restricted_for_use.fasta',
 'candida_parapsilosis': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20180108_FAH18647/barcode06/length_res

In [54]:
sub_reads_fn = {}
for key, value in fn_subsampling.items():
    species = key
    in_fn = value
    out_fn = os.path.join(MC_READ_DIR, F'{species}.{n_reads}.fasta')
    subsamplereads(in_fn, out_fn, n_reads)
    sub_reads_fn[species] = out_fn

:(check one reformat.sh samplereadstarget=15000 in=/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171103_FAH15473/barcode10/length_restricted_for_use.fasta out=/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/penicillium_chrysogenum.15000.fasta!!

:(check one reformat.sh samplereadstarget=15000 in=/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171207_FAH18654/barcode12/length_restricted_for_use.fasta out=/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/aspergillus_flavus.15000.fasta!!

:(check one reformat.sh samplereadstarget=15000 in=/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171103_FAH15473/barcode07/length_restricted_for_use.fasta out=/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/aspergillus_niger.15000.fasta!!

:(check one reformat.sh samplereadstarget=15000 in=/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20180108_FAH18647/barcode11/length_restricted_for_use.fasta out=/media/MassStorage

### Map with minimap against both databases

In [56]:
def minimapmapping(fasta_fn, ref_fn, out_fn):
    command = F"minimap2 -x map-ont -t 6 {ref_fn} {fasta_fn} -o {out_fn}"
    out = subprocess.getstatusoutput(command)
    if out[0] == 0:
        print(F":)Completed {command}\n")
    else:
        print(F":(check one {command}!!\n")

In [57]:
dbases_fn = {}
for x in [sub_db_fn, new_db_fn]:
    dbases_fn[x] = os.path.join(OUT_DIR, os.path.basename(x).replace('.fasta', '').replace('.','_'))
    if not os.path.exists(dbases_fn[x]):
        os.mkdir(dbases_fn[x])
dbases_fn

{'/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.subdb.fasta': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb',
 '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.db.fasta': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_db'}

In [58]:
db_fn = sub_db_fn
sub_db_mapping_fn = {}
for species, fasta_fn in sub_reads_fn.items():
    tmp_out = dbases_fn[db_fn]
    db_name = os.path.basename(db_fn).replace('.fasta', '')
    out_fn = os.path.join(tmp_out, F"{db_name}.{species}.minimap2.paf")
    sub_db_mapping_fn[species] = out_fn
    minimapmapping(fasta_fn, db_fn, out_fn)

:)Completed minimap2 -x map-ont -t 6 /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.subdb.fasta /media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/penicillium_chrysogenum.15000.fasta -o /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.penicillium_chrysogenum.minimap2.paf

:)Completed minimap2 -x map-ont -t 6 /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.subdb.fasta /media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/aspergillus_flavus.15000.fasta -o /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.aspergillus_flavus.minimap2.paf

:)Completed minimap2 -x map-ont -t 6 /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.subdb.fasta /media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/aspergillus_niger.15000.fasta -o /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.aspergillus_niger.minimap2.paf

:)Completed minimap2 -x map-ont -t 6 /media/MassStorage/tmp/TE/honou

In [59]:
sub_db_mapping_fn

{'penicillium_chrysogenum': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.penicillium_chrysogenum.minimap2.paf',
 'aspergillus_flavus': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.aspergillus_flavus.minimap2.paf',
 'aspergillus_niger': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.aspergillus_niger.minimap2.paf',
 'pichia_kudriavzevii': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.pichia_kudriavzevii.minimap2.paf',
 'pichia_membranifaciens': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.pichia_membranifaciens.minimap2.paf',
 'candida_albicans': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.candida_albicans.minimap2.paf',
 'candida_parapsilosis': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.candida_parapsilosis.minimap2.paf',
 'candida_orthopsilosis': '/media/MassStorage/tmp/T

In [60]:
db_fn = new_db_fn
new_db_mapping_fn = {}
for species, fasta_fn in sub_reads_fn.items():
    tmp_out = dbases_fn[db_fn]
    db_name = os.path.basename(db_fn).replace('.fasta', '')
    out_fn = os.path.join(tmp_out, F"{db_name}.{species}.minimap2.paf")
    new_db_mapping_fn[species] = out_fn
    minimapmapping(fasta_fn, db_fn, out_fn)

:)Completed minimap2 -x map-ont -t 6 /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.db.fasta /media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/penicillium_chrysogenum.15000.fasta -o /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_db/gsref.db.penicillium_chrysogenum.minimap2.paf

:)Completed minimap2 -x map-ont -t 6 /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.db.fasta /media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/aspergillus_flavus.15000.fasta -o /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_db/gsref.db.aspergillus_flavus.minimap2.paf

:)Completed minimap2 -x map-ont -t 6 /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.db.fasta /media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/aspergillus_niger.15000.fasta -o /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_db/gsref.db.aspergillus_niger.minimap2.paf

:)Completed minimap2 -x map-ont -t 6 /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.d

### Look at mapping results

In [61]:
def mapping_results(fn, species):
    min_header = ['qseqid', 'qlen', 'qstart', 'qstop', 'strand', 'tname', 'tlen', 'tstart', 'tend', 'nmatch', 'alen', 'mquality']
    tmp_df = pd.read_csv(fn, sep='\t', header = None, usecols=[x for x in range(0,12)], names=min_header)
    sub_df = tmp_df[tmp_df['mquality'] == tmp_df.groupby('qseqid')['mquality'].transform(max)].reset_index(drop=True)
    sub_df = sub_df[tmp_df['nmatch'] == tmp_df.groupby('qseqid')['nmatch'].transform(max)].reset_index(drop=True)
    hit_series = pd.Series(sub_df.groupby('tname')['mquality'].count().tolist()/sub_df.groupby('tname')['mquality'].count().sum(),
                      sub_df.groupby('tname')['mquality'].count().index)
    hit_series.sort_values(ascending=False, inplace=True)
    print('##########\n')
    print(F"This was the query species: {species}\n")
    print(F"These are the results:")
    print(hit_series,'\n')

In [62]:
###this is running the reads against the full database
for species, hit_fn in new_db_mapping_fn.items():
    mapping_results(hit_fn, species)

/home/tavish/anaconda3/envs/honours1/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


##########

This was the query species: penicillium_chrysogenum

These are the results:
tname
penicillium_chrysogenum          0.987255
aspergillus_niger                0.002048
aspergillus_flavus               0.001972
aspergillus_unidentified         0.001745
kluyveromyces_marxianus          0.000986
kluyveromyces_unidentified       0.000986
zymoseptoria_tritici             0.000759
tuber_brumale                    0.000531
cortinarius_globuliformis        0.000531
cladophialophora_unidentified    0.000379
rhodotorula_mucilaginosa         0.000379
kodamaea_ohmeri                  0.000228
yamadazyma_scolyti               0.000152
puccinia_striiformis-tritici     0.000152
scedosporium_boydii              0.000152
asteroma_ccl060                  0.000152
clavispora_lusitaniae            0.000152
zygoascus_hellenicus             0.000152
dothiorella_vidmadera            0.000152
yamadazyma_mexicana              0.000076
blastobotrys_proliferans         0.000076
candida_metapsilosis    

In [63]:
###this is running against a database that have ['Candida_orthopsilosis', 'Candida_metapsilosis', 'Aspergillus_niger'] deleted
for species, hit_fn in sub_db_mapping_fn.items():
    mapping_results(hit_fn, species)

/home/tavish/anaconda3/envs/honours1/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


##########

This was the query species: penicillium_chrysogenum

These are the results:
tname
penicillium_chrysogenum          0.990067
aspergillus_unidentified         0.001779
aspergillus_flavus               0.001631
kluyveromyces_unidentified       0.000964
kluyveromyces_marxianus          0.000889
zymoseptoria_tritici             0.000593
tuber_brumale                    0.000519
cortinarius_globuliformis        0.000445
rhodotorula_mucilaginosa         0.000296
cladophialophora_unidentified    0.000296
pyrenophora_tritici-repentis     0.000148
yamadazyma_scolyti               0.000148
asteroma_ccl060                  0.000148
blastobotrys_proliferans         0.000148
candida_unidentified             0.000148
clavispora_lusitaniae            0.000148
debaryomyces_unidentified        0.000148
dothiorella_vidmadera            0.000148
fusarium_oxysporum               0.000148
zygoascus_hellenicus             0.000148
yamadazyma_mexicana              0.000148
kodamaea_ohmeri         

##########

This was the query species: candida_metapsilosis

These are the results:
tname
candida_parapsilosis             0.959939
candida_albicans                 0.007885
candida_unidentified             0.007376
meyerozyma_guillermondii         0.005469
debaryomyces_unidentified        0.004324
kodamaea_ohmeri                  0.001780
yamadazyma_scolyti               0.001780
wickerhamomyces_anomalus         0.001780
clavispora_lusitaniae            0.001653
yamadazyma_mexicana              0.001272
kluyveromyces_unidentified       0.000763
blastobotrys_proliferans         0.000763
galactomyces_geotrichum          0.000763
kluyveromyces_marxianus          0.000763
fusarium_oxysporum               0.000509
zymoseptoria_tritici             0.000382
pichia_membranifaciens           0.000382
saccharomyces_cerevisiae         0.000382
zygoascus_hellenicus             0.000254
oculimacula_yallundae-ccl031     0.000254
cladophialophora_unidentified    0.000254
tuber_brumale              